# Generate prompts from raw stimuli and instructions for different measurements & different tasks
Author: Hening Wang
Date: 28.08.2023

## Generate prompts for Rating task

In [1]:
# Import relevant modules
import pandas as pd
import numpy as np


In [14]:
phenomenon = 'Irony'
# Read the instructions of rating
file_path_instructions_rating = f'raw_stimuli/{phenomenon}Instructions_Rating.txt'

with open(file_path_instructions_rating, 'r') as file:
    instructions_rating = file.read()

print(instructions_rating)

# Read options from the file

file_path_options = f'raw_stimuli/{phenomenon}Options.txt'

# Reading the content of options into a DataFrame
column_names = ['item_number', 'option_number', 'option_text']
content_options = pd.read_csv(file_path_options, sep='\t', names=column_names)

# Create a list of lists of options for each item
options = []
for item_number in content_options.item_number.unique():
    options.append(content_options[content_options.item_number == item_number].option_text.tolist())

print(options)

# Read key from the file
key_options = pd.read_csv(f'raw_stimuli/{phenomenon}KeySimple.txt', sep='\t', names=["phenomenon",'item_number', "option_number",'content'])

# Find the key for each item
keys_items_pairs = []
for i, row in key_options.iterrows():
    if row.content == 'Correct':
        keys_items_pairs.append((row.item_number, row.option_number))

# Arrange the keys_items_pairs in the upscent order of item_number
keys_items_pairs.sort(key=lambda x: x[0])

# Reading the scenario content from the file
file_path_scenario = f'raw_stimuli/{phenomenon}Scenarios.txt'
scenarios = pd.read_csv(file_path_scenario, sep='\t', names=['item_number', 'scenario'])

# Get true answer with options and key

## Get true answer with key item pairs
true_answer = []
for item_number, option_number in keys_items_pairs:
    matching_option = content_options[
        (content_options['item_number'] == item_number) & 
        (content_options['option_number'] == option_number)
    ]['option_text']
    matching_option = "Explanation: " + matching_option
    true_answer.extend(matching_option.tolist())

print(true_answer)

Task: You will read short stories that describe everyday situations. Each story will be followed by an explanation of why one character in that scenario responds in a certain way. Read each story and the associated explanation. Your task is to decide how plausible is that explanation. The degree varies from "very implausible", "implausible", "at chance", "plausible", "very plausible". Answer only with one degree.
[['The dress is fitting well.', 'Lara needs to get a dress of a shorter length.', 'Simon does not like the color of this dress.', 'Simon has to get back to work.'], ['Shane was paying attention when Christian was talking.', 'Shane was not paying attention when Christian was talking.', "Shane's hearing improved greatly.", 'Shane needs to see an eye doctor.'], ['Phil is a great basketball player.', "Phil's basketball skills could be better.", "Phil's blocking is impressive.", 'Phil is a great student.'], ['Patrick is very cautious.', 'Patrick is careless.', 'Patrick is allergic 

In [15]:
# Combine instructions, scenarios and true anwsers together
new_prompts = []
for i in range(len(true_answer)):
    new_prompts.append(instructions_rating + " " + "Scenario: " + scenarios.scenario[i] + " " + true_answer[i])

# Create a new DataFrame for the new prompts for rating task
dict_prompts_rating = {'item_number': scenarios.item_number,
                       'prompt': new_prompts,
                       'true_answer_key': [keys_items_pair[1] for keys_items_pair in keys_items_pairs],
                       'true_answer': true_answer,
                       'scenarios': scenarios.scenario,
                       'options': options}

df_prompts_rating = pd.DataFrame(dict_prompts_rating)
df_prompts_rating.to_csv(f"{phenomenon}_prompts_Rating.csv", index=False)

## Generate prompts for free production

In [17]:
# Read the instructions for free production
file_path_free = f'raw_stimuli/{phenomenon}Instructions_Free.txt'

with open(file_path_free, 'r') as file:
    instructions_free = file.read()

# Copf df_prompts as df_prompts_free
df_prompts_free = df_prompts.copy()

# Combine instructions and scenarios together
new_prompts = []
for i in range(len(scenarios.scenario)):
    new_prompts.append(instructions_free + " " + "Scenario: " + scenarios.scenario[i])

dict_prompts_free = {'item_number': scenarios.item_number,
                       'prompt': new_prompts,
                       'true_answer_key': [keys_items_pair[1] for keys_items_pair in keys_items_pairs],
                       'true_answer': true_answer,
                       'scenarios': scenarios.scenario,
                       'options': options}

df_prompts_free = pd.DataFrame(dict_prompts_free)
df_prompts_free.to_csv(f"{phenomenon}_prompts_Free.csv", index=False)

